
**Домашнее задание к вебинару 13. Transformers**

Решить задачу парафраза.

In [1]:
!pip install datasets
!pip install transformers
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.5 MB/s 
     |████████████████████████████████| 141 kB 67.9 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
     |████████████████████████████████| 212 kB 66.4 MB/s 
     |████████████████████████████████| 596 kB 66.3 MB/s 
     |████████████████████████████████| 127 kB 72.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 w

In [2]:
import pandas as pd
from datasets import load_dataset

corpus_train = load_dataset('merionum/ru_paraphraser', data_files='train.jsonl')
corpus_test = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Using custom data configuration merionum--ru_paraphraser-4a63244a00d42f51


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4a63244a00d42f51/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration merionum--ru_paraphraser-4ab1b249bc6d5bf7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4ab1b249bc6d5bf7/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
df_train = pd.DataFrame(corpus_train['train'])
df_train['text'] =  list(zip(df_train['text_1'], df_train['text_2']))
df_train = df_train[:2000]
df_train.head()

,id,id_1,id_2,text_1,text_2,class,text
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,(Полицейским разрешат стрелять на поражение по...
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,(Право полицейских на проникновение в жилище р...
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,(Президент Египта ввел чрезвычайное положение ...
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,(Вернувшихся из Сирии россиян волнует вопрос т...
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,(В Москву из Сирии вернулись 2 самолета МЧС с ...


In [4]:
df_test = pd.DataFrame(corpus_test['train'])
df_test['text'] =  list(zip(df_test['text_1'], df_test['text_2']))
df_test = df_test[:500]
df_test.head()

,id,id_1,id_2,text_1,text_2,class,text
0,25349,34420,34421,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,-1,"(Цены на нефть восстанавливаются, Парламент Сл..."
1,25360,34438,34439,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,-1,"(""Гоголь-центр"" покажет видеозапись скандально..."
2,25382,34475,34476,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,-1,(Агент: РФС вновь задерживает зарплату Фабио К...
3,25423,34494,34519,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,-1,(День Победы в Москве обещает выдаться облачны...
4,25430,34531,34532,Посол РФ в США: Россия будет бороться с попытк...,Правительство запланировало заработать на лоте...,-1,(Посол РФ в США: Россия будет бороться с попыт...


In [5]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [6]:
text1 = df_test['text_1'][0]
text2 = df_test['text_2'][0]
tokenizer(text1, text2)

{'input_ids': [101, 59713, 1469, 20264, 26631, 5052, 102, 28362, 27443, 57650, 37777, 12678, 5576, 1758, 50938, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df_train['class'])

df_train['class'] = le.transform(df_train['class']) #array([1, 1, 1, ..., 0, 0, 0])
df_test['class'] = le.transform(df_test['class'])

In [8]:
df_train.rename(columns = {'class': 'label'}, inplace = True)
df_test.rename(columns = {'class': 'label'}, inplace = True)

In [9]:
from datasets import Dataset, DatasetDict

data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(df_train[['text_1', 'text_2', 'label']])),
    'test': Dataset.from_pandas(pd.DataFrame(df_test[['text_1', 'text_2', 'label']]))
})

In [10]:
data['train']

Dataset({
    features: ['text_1', 'text_2', 'label'],
    num_rows: 2000
})

In [11]:
def tokenize_function(examples):
  return tokenizer(examples["text_1"], examples["text_2"], padding="max_length", truncation=True, max_length = 22)
  # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length = 22)

In [12]:
tokenized_datasets = data.map(tokenize_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

  0%|          | 0/1 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [13]:
num_labels = df_test['label'].nunique()
num_labels

3

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels, ignore_mismatched_sizes=True).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-base-cased-dp-paraphrase-detection and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

batch_size = 128

# Определяем аргументы
args = TrainingArguments(
    'classification',
    evaluation_strategy = "epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    # weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

# args = TrainingArguments(output_dir="test_trainer")

In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [18]:
from datasets import load_metric

metric = load_metric("accuracy")

In [19]:
import numpy as np

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics = compute_metrics,
    # data_collator=data_collator, # в классификации не требуется
    # tokenizer=tokenizer,
)

In [21]:
for param in model.parameters():
    param.requires_grad = True

In [22]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [23]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.151413,0.264000
2,No log,1.117959,0.256000
3,No log,1.093112,0.258000
4,No log,1.076818,0.258000
5,No log,1.070140,0.266000
6,No log,1.063034,0.268000
7,No log,1.045939,0.322000
8,No log,1.028200,0.412000
9,No log,1.017320,0.486000
10,No log,0.999497,0.566000


TrainOutput(global_step=800, training_loss=0.7699198913574219, metrics={'train_runtime': 433.5194, 'train_samples_per_second': 230.67, 'train_steps_per_second': 1.845, 'total_flos': 1130565466800000.0, 'train_loss': 0.7699198913574219, 'epoch': 50.0})

In [24]:
trainer.evaluate()

{'epoch': 50.0,
 'eval_accuracy': 0.656,
 'eval_loss': 0.8193555474281311,
 'eval_runtime': 0.6315,
 'eval_samples_per_second': 791.822,
 'eval_steps_per_second': 6.335}

In [25]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=-1)
metric.compute(predictions=predictions, references=labels)

{'accuracy': 0.656}

In [26]:
text_1 = df_test['text_1'][3]
text_2 = df_test['text_2'][3]
text_1, text_2

('День Победы в Москве обещает выдаться облачным',
 'Любляна отпразднует День Победы вместе с Москвой')

In [27]:
tokens = tokenizer(text_1, text_2,  padding="max_length", truncation=True, max_length = 22, return_tensors='pt').to(device)

In [28]:
tokens = {k: v for k, v in tokens.items()}

In [29]:
with torch.no_grad():
    pred = model(**tokens)

In [30]:
print(f'Истина: {df_test["label"][3]}, \nПредсказание:{pred.logits.argmax(dim=-1)[0].cpu().numpy()}')

Истина: 0, 
Предсказание:0


В целом модель неплохо отработала контекст, но смысловое значение изменено